In [1]:
import numpy as np
import pandas as pd
import fm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [2]:
cities = ['city_' + x for x in ['A','B','C','D','E'] ]
def parse(x):
    x = x[:4] + ' '+ x[4:6] + ' '+x[6:]
    return datetime.strptime(x, '%Y %m %d')
# city = pd.read_csv('train_data/' + c + '/infection.csv',parse_dates = ['date'], date_parser=parse)

In [3]:
def read_infection(c):
    ifct = pd.read_csv('train_data/' + c + '/infection.csv',header = None)
    ifct.columns = ['city', 'region', 'date', 'infections']
    return ifct
def read_data():
    for c in cities:
        yield read_infection(c)

In [31]:
A,B,C,D,E = read_data()
date_list = A[A.region==0].date.values

In [27]:
def read_weather(c):
    wthr = pd.read_csv('train_data/' + c + '/weather.csv',header = None)
    wthr.columns = ['date','hour','temp','humidity','fx','fs','fl','w']
    wthr['humidity'] = wthr['humidity'].fillna(method="ffill")
    wthr["humidity"] = [int(x[:-1])*0.01 for x in wthr["humidity"].values]
    wthr.drop(['fx','fs','fl','w'],axis = 1,inplace = True)
    df = pd.DataFrame(date_list)
    df.columns = ['date']
    for i in range(24):
        df["temp_"+str(i)] = wthr[wthr.hour==i]["temp"].values
        df["humidity_"+str(i)] = wthr[wthr.hour==i]["humidity"].values
    return df

In [32]:
A = pd.merge(A,read_weather(cities[0]),on="date")
B = pd.merge(B,read_weather(cities[1]),on="date")
C = pd.merge(C,read_weather(cities[2]),on="date")
D = pd.merge(D,read_weather(cities[3]),on="date")
E = pd.merge(E,read_weather(cities[4]),on="date")

In [57]:
def read_migration(c):
    m = pd.read_csv('train_data/' + c + '/migration.csv',header = None)
    m.columns = ["date","from","to","index"]
    f = m[m["from"]==c[-1]].groupby("date").sum()
    f.columns = ["out_index"]
    t = m[m["to"]==c[-1]].groupby("date").sum()
    t.columns = ["in_index"]
    return f,t

In [60]:
def merge_mig(df,index):
    f,t = read_migration(cities[index])
    df = pd.merge(df,f,on="date")
    df = pd.merge(df,t,on="date")
    return df

In [65]:
A = merge_mig(A,0)
B = merge_mig(A,1)
C = merge_mig(A,2)
D = merge_mig(A,3)
E = merge_mig(A,4)

In [64]:
A = pd.read_csv("A.csv")
B = pd.read_csv("B.csv")
C = pd.read_csv("C.csv")
D = pd.read_csv("D.csv")
E = pd.read_csv("E.csv")

In [68]:
A.to_csv("A.csv",index=False)
B.to_csv("B.csv",index=False)
C.to_csv("C.csv",index=False)
D.to_csv("D.csv",index=False)
E.to_csv("E.csv",index=False)